In [1]:
import sys
import time
import pandas as pd
import numpy as np

# QCGPU Imports
import qcgpu

# ProjectQ Imports
from projectq import MainEngine
import projectq.ops as ops
from projectq.backends import Simulator
from projectq.types import Qureg



In [12]:
def bench_qcgpu(n, depth):
    state = qcgpu.State(n)

    h = qcgpu.gate.h()
    x = qcgpu.gate.x()
    sqrt_x = qcgpu.gate.sqrt_x()

    print('started')
    start = time.time()

    for level in range(depth):
        for q in range(n):
    
            state.apply_gate(h, q)
            state.apply_gate(sqrt_x, q)

            if q != 0:
                state.apply_controlled_gate(x, q, 0)
        
    runtime = time.time() - start
    print('ended: ', runtime)
    return runtime
    
def bench_projectq(n, depth):
    eng = MainEngine(backend=Simulator(gate_fusion=True), engine_list=[])
    qbits = eng.allocate_qureg(n)

    print('started')
    start = time.time()

    for level in range(depth):
        print(depth-level)
        for q in qbits:
            ops.H | q
            ops.SqrtX | q
            if q != qbits[0]:
                ops.CNOT | (q, qbits[0])

    runtime = time.time() - start
    print('ended: ', runtime)

    for q in qbits:
        ops.Measure | q
    eng.flush()
    return runtime


In [10]:
bench_qcgpu(28,100)

started
('ended: ', 202.95192885398865)


202.95192885398865

In [13]:
bench_projectq(28,100)

started
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
('ended: ', 971.3228080272675)


971.3228080272675